In [40]:
%serialconnect


Connecting to --port=/dev/ttyUSB0 --baud=115200 
Traceback (most recent call last):
  File "main.py", line 27, in <module>
  File "UDPblackviewphone.py", line 19, in connectActivePhone
KeyboardInterrupt: 
MicroPython v1.17 on 2021-09-02; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [17]:
%sendtofile config.txt

pinled           2
deviceletter     @
connection0      JulianS9,beckaaaa,192.168.43.1,9042
analogreadpin    4
analogreadrate   100


Sent 5 lines (131 bytes) to config.txt.


In [18]:
%sendtofile --source UDPblackviewphone.py
%sendtofile --source utils.py
# Use deviceletter=E for second ascii stream (to decode in the Android code)

Sent 32 lines (969 bytes) to utils.py.


In [41]:
%sendtofile main.py

from machine import Pin, ADC, TouchPad
import time, socket, urandom
from UDPblackviewphone import connectActivePhone

fconfig = dict(x.strip().split(None, 1)  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
deviceletter = fconfig["deviceletter"]
c = fconfig["connection0"].split(",")
hotspots = { c[0].encode(): (c[1].encode(), c[2], int(c[3])) }
androidipnumber = c[2]
portnumber = int(c[3])
print("deviceletter", deviceletter)

touchpad0 = TouchPad(Pin(4))
touchpad1 = TouchPad(Pin(13))
touchpadreadrate = 100
tstamptouchpadread = 0

nbs = bytearray("Nt00000000s000000r000000\n")
mnbs = memoryview(nbs)

while not connectActivePhone(pinled, hotspots):
    for j in range(3):
        for i in range(7):
            pinled.value(i%2)
            time.sleep_ms(100)
        time.sleep_ms(800)

while True:
    try:
        print("making socket")
        ss = socket.socket()
        ss.settimeout(1)
        print("socket is", ss)
        ss.connect(socket.getaddrinfo(androidipnumber, portnumber)[0][-1])
        s = ss.makefile('rwb', 0)
        print(s.readline())
        s.write(b"%c%c%c%c"%(deviceletter,deviceletter,deviceletter,deviceletter))
        dwrite = s.write
        while True:
            tstamp = time.ticks_ms()
            if tstamp > tstamptouchpadread:
                tstamptouchpadread = tstamp + touchpadreadrate
                mnbs[2:10] = b"%08X" % tstamp
                mnbs[11:17] = b"%06X" % touchpad0.read()
                mnbs[18:24] = b"%06X" % touchpad1.read()
                dwrite(nbs)
                
    except OSError as e:
        msg = str(e)
        if e.args[0] == 118:
            msg = "wifi not connected"
        elif e.args[0] == 23:
            msg = "socket error"
        print("OSError", msg)
        time.sleep(2)


Sent 56 lines (1800 bytes) to main.py.
